## Define a Batch Attribute Group (via SDK)

In [5]:
from snowplow_signals import Signals
from dotenv import load_dotenv
import os

load_dotenv()

sp_signals = Signals(
    api_url=os.environ["SP_API_URL"],
    api_key=os.environ["SP_API_KEY"],
    api_key_id=os.environ["SP_API_KEY_ID"],
    org_id=os.environ["SP_ORG_ID"],
)


### Define attributes

This block creates attribute definitions including the logic how it should be calculated (it's filters and aggregation).

In [ ]:
from snowplow_signals import (
    Attribute,
    Event,
    Criterion,
    Criteria,
    EventProperty,
    EntityProperty,
)

sum_transaction_value_ltv = Attribute(
    name="sum_transaction_value_ltv",
    type="float",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow.ecommerce",
            name="snowplow_ecommerce_action",
            version="1-0-2",
        )
    ],
    aggregation="sum",
    property=EntityProperty(
        vendor="com.snowplowanalytics.snowplow.ecommerce",
        name="transaction",
        major_version=1,
        path="revenue",
    ),
    criteria=Criteria(
        all=[
            Criterion.eq(
                EventProperty(
                    vendor="com.snowplowanalytics.snowplow.ecommerce",
                    name="snowplow_ecommerce_action",
                    major_version=1,
                    path="type",
                ),
                "transaction",
            )   
        ]
    ),
)

last_product_category_purchased = Attribute(
    name="last_product_category_purchased",
    type="string",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow.ecommerce",
            name="snowplow_ecommerce_action",
            version="1-0-2",
        )
    ],
    aggregation="last",
    property=EntityProperty(
        vendor="com.snowplowanalytics.snowplow.ecommerce",
        name="product",
        major_version=1,
        path="category",
    ),
)


### Wrapping the attribute in a group

All attributes need to be included in attribute groups that can be considered as "tables" of attributes.

Attribute groups are immutable and versioned.

In [ ]:
from snowplow_signals import BatchAttributeGroup, user_id

attribute_group = BatchAttributeGroup(
    name="user_activity",
    version=1,
    attribute_key=user_id,
    attributes=[
        sum_transaction_value_ltv, last_product_category_purchased
    ],
    owner="user@company.com",
)

### Testing the attribute group

Execute the attribute group on the last one hour of data from the atomic events table to verify that it works correctly.

In [ ]:
sp_signals.test(attribute_group=attribute_group)

# Unpublish the attribute group

Unpublish the attribute group to make it available for the Batch Engine to process it.

In [ ]:
sp_signals.unpublish([attribute_group])

# Follow the Batch Engine [Tutorial](https://deploy-preview-1197--snowplow-docs.netlify.app/tutorials/signals-batch-engine/start/)
You will need to go through the following steps:
- use the Batch Engine CLI tool to generate the dbt project and models
- configure your dbt setup / test / make modifications if needed / set up a regular dbt run schedule
- use the Batch Engine CLI to define the Batch Source and to publish your attribute group for the sync to begin

# Wrap the attribute group in a service

In [ ]:
from snowplow_signals import Service
service = Service(
    name="user_activity",
    attribute_groups=[attribute_group],
    owner="user@company.com",
)

# Retrieve attributes for the service

In [ ]:
service.get_attributes(
    signals=sp_signals,
    attribute_key="user_id",
    identifier="user_id_1",
)